1. poetry add aws-glue-sessions
2. Run install-glue-kernel on Terminal
3. Install awsglue library: poetry add git+https://github.com/awslabs/aws-glue-libs.git
3. Run notebook jupyter
4. Copy the URL from running Jupyter
5. Open kernels -> Select Another Kernel -> Existing Jupyter Server
4. Select Glue Spark kernel
5. Settings -> jupyter disable auto start
6. Create a IAM Role for Glue with the policy: AwsGlueSessionUserRestrictedNotebookServiceRole or AWSGlueServiceRole 
7. Create credentials and save it at ~/.aws/credentials as following
    ```
    [default]
    aws_access_key_id = TEXT_HERE
    aws_secret_access_key = TEXT_HERE
    glue_iam_role=arn:aws:iam::<ACCOUNT_ID>:role/glue-interative-sessions-role
    session_id_prefix=etl-glue
    ```

8. Create a config file at ~/.aws/config and enter the following info
    ```
    [default]
    region = us-east-2
    ```
9. Create a new bucket to store raw data and upload data (s3://{BUCKET_NAME}{inputs})
10. Create a policy for Glue read data from S3 inputs folder 

Links:
https://docs.aws.amazon.com/glue/latest/dg/aws-glue-programming-intro-tutorial.html
https://github.com/awslabs/aws-glue-libs

**Policy for Glue access Bikes bucket**

```{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "VisualEditor0",
			"Effect": "Allow",
			"Action": [
				"s3:PutObject",
				"s3:GetObject",
				"s3:UntagResource",
				"s3:TagResource",
				"s3:CreateBucket",
				"s3:ListBucket",
				"s3:DeleteObject",
				"s3:DeleteBucket"
			],
			"Resource": "arn:aws:s3:::{BUCKET_NAME}/*"
		}
	]
}
```

In [85]:
%session_id_prefix etl-bikes
%iam_role arn:aws:iam::078090784717:role/glue-interative-sessions-role 
%region us-east-1
%glue_version 4.0
%idle_timeout 15
%worker_type G.1X
%number_of_workers 2
%%configure
{
  "max_retries": "0" # The number of times to retry the job in case of a failure.
# "max_concurrent_runs": "3" # The maximum number of concurrent runs allowed for a job.
# "script_location": "s3://new-folder-here" # The Amazon S3 path to a script that executes a job.
#  "--extra-files": "s3://path/to/additional/files/" # The Amazon S3 paths to additional files, such as configuration files that AWS Glue copies to the working directory of your script before executing it.
#  "--enable-glue-datacatalog": "true"  # Enables you to use the AWS Glue Data Catalog as an Apache Spark Hive metastore.
#  "--enable-metrics": "false" # Enables the collection of metrics for the job.
#  "--enable-continuous-cloudwatch-log": "true" # Enables the continuous logging of the job.
#  "timeout": "30" # Determines the maximum amount of time that the Spark session should wait for a statement to complete before terminating it.
#  "--enable-auto-scaling": "true"  # Determines whether or not to use auto-scaling.
}  

Setting session ID prefix to etl-bikes


You are already connected to a glueetl session etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current iam_role is arn:aws:iam::078090784717:role/glue-interative-sessions-role
iam_role has been set to arn:aws:iam::078090784717:role/glue-interative-sessions-role.


You are already connected to a glueetl session etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1


...

Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 2
Setting new number of workers to: 2


You are already connected to a glueetl session etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


The following configurations have been updated: {'max_retries': '0'}


In [87]:
%additional_python_modules  typing-extensions, typing-inspect, pydantic, boto3, botocore, polars, awswrangler, pyathena


You are already connected to a glueetl session etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Additional python modules to be included:
typing-extensions
typing-inspect
pydantic
boto3
botocore
polars
awswrangler
pyathena


In [88]:
#%etl # Changes the session type to AWS Glue ETL.
#%extra_py_files # Comma separated list of additional Python files from Amazon S3.
#%%sql # Run SQL queries
#%matplot # Display Matplotlib plots

In [25]:
import boto3
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import sys

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

#args = getResolvedOptions(sys.argv, ["JOB_NAME"])
job.init('Bikes-ETL')

In [90]:
%status

Session ID: etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4
Status: READY
Role: arn:aws:iam::078090784717:role/glue-interative-sessions-role
CreatedOn: 2024-09-29 18:41:16.712000+00:00
GlueVersion: 4.0
Session Type: glueetl
Idle Timeout: 15
Timeout: 2880
Tags: {'owner': 'AIDARELUSZ7G54LO3PEET'}
Worker Type: G.1X
Number of Workers: 2
Region: us-east-1
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
--max_retries 0
--additional-python-modules typing-extensions,typing-inspect,pydantic,boto3,botocore,polars,awswrangler,pyathena
Arguments Passed: ['--glue_kernel_version: 1.0.7', '--enable-glue-datacatalog: true', '--max_retries: 0', '--additional-python-modules: typing-extensions,typing-inspect,pydantic,boto3,botocore,polars,awswrangler,pyathena']


### Create a Glue Catalog from S3 csv files

https://docs.aws.amazon.com/glue/latest/dg/start-data-catalog.html

In [26]:
database_name = "bikes_catalog"
table_name = "bikes_table"

glue_client = boto3.client('glue')

def load_data_from_s3_as_DF(bucket_name, prefix,  key):
    import awswrangler as wr
    import polars as pl
    from awsglue.dynamicframe import DynamicFrame
    
    session = boto3.Session()
    df_pandas = wr.s3.read_csv(f"s3://{bucket_name}/{prefix}/{key}",
                          boto3_session=session)
    df_spark = spark.createDataFrame(pl.from_pandas(df_pandas).to_pandas())
    return DynamicFrame.fromDF(df_spark, glueContext)

def database_exists(database_name):
    try:
        glue_client.get_database(Name=database_name)
        return True
    except glue_client.exceptions.EntityNotFoundException:
        return False
    
if not database_exists(database_name):
    glue_client.create_database(
        DatabaseInput={
            'Name': database_name,
            'Description': 'Database for bike data'
        }
    )
    
def add_table(database_name, table_name, dynamic_frame):
    glue_client.create_table(
        DatabaseName=database_name,
        TableInput={
            'Name': table_name,
            'StorageDescriptor': {
                'Columns': [
                    {
                        'Name': field.name,
                        'Type': field.dataType.typeName()
                    }
                    for field in dynamic_frame.schema().fields
                ],
                'Location': f"s3://bikes-glue/inputs/", 
                'Parameters': {
                    'EXTERNAL': 'TRUE',
                    'classification': 'csv',
                    'columnsOrdered': 'true',
                    'compressionType': 'none',
                    'delimiter': ',',
                    'skip.header.line.count': '1',
                    'typeOfData': 'file'
                }
            }
        }   
    )
    
def table_exists(database_name, table_name):
    try:
        glue_client.get_table(DatabaseName=database_name, Name=table_name)
        return True
    except glue_client.exceptions.EntityNotFoundException:
        return False
    
tables = ['202406-divvy-tripdata']
for table_name in tables:
    if not table_exists(database_name, table_name):
        dynamic_frame = load_data_from_s3_as_DF("bikes-glue", "inputs", f"{table_name}.csv")
        add_table(database_name, table_name, dynamic_frame)

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


### Extract data from a source

In [27]:
def extract(table_name):
    # connect to tables in the AWS Glue Data Catalog
    S3bucket_node1 = glueContext.create_dynamic_frame.from_catalog(
        database=database_name, 
        table_name=table_name, 
        transformation_ctx="S3bucket_node1" # support job bookmarks
    )
    return S3bucket_node1


### Transform data with AWS Glue

In [28]:
def transform(frame):
    ApplyMapping_node2 = ApplyMapping.apply(
        frame=frame,
        mappings=[
            ("ride_id", "string", "ride_id", "string"),
            ("rideable_type", "string", "rideable_type", "string"),
            ("started_at", "string", "started_at", "timestamp"),
            ("ended_at", "string", "ended_at", "timestamp"),
            ("start_station_name", "string", "start_station_name", "string"),
            ("start_station_id", "int", "start_station_id", "string"),
            ("end_station_name", "string", "end_station_name", "string"),
            ("end_station_id", "int", "end_station_id", "string"),
            ("start_lat", "double", "start_lat", "double"),
            ("start_lng", "double", "start_lng", "double"),
            ("end_lat", "double", "end_lat", "double"),
            ("end_lng", "double", "end_lng", "double"),
            ("member_casual", "string", "member", "string"),
        ],
        transformation_ctx="ApplyMapping_node2",
    )
    return ApplyMapping_node2

### Load data into a target

In [32]:
def load(frame, output_dir="s3://aws-glue-transformed/"):
    
    # if bucket does not exist, create it
    bucket_name = output_dir.split("/")[2]
    s3 = boto3.resource('s3')
    if bucket_name not in [bucket.name for bucket in s3.buckets.all()]:
        s3.create_bucket(Bucket=bucket_name)
    
    S3bucket_node3 = glueContext.write_dynamic_frame.from_options(
        frame=frame,
        connection_type="s3",
        format="glueparquet",
        connection_options={"path": output_dir, "partitionKeys": []},
        format_options={"compression": "gzip"},
        transformation_ctx="S3bucket_node3",
    )
    return S3bucket_node3

### Execute ETL with Glue

In [33]:
frame = extract("202406-divvy-tripdata")
frame = transform(frame)
frame = load(frame)
frame.printSchema()

root


Manually conclude job lifecycle 

In [30]:
job.commit()

Manually stop Glue Sessions

In [92]:
%stop_session

Stopping session: etl-bikes-6fd446c9-db07-4d34-a6c7-2f68ea3333a4
Stopped session.
